In [1]:

import os
import re
import random
import pprint
import mne
from mne.io.edf.edf import RawEDF
import numpy as np
from datetime import datetime
import pandas as pd
import pywt
from scipy.stats import skew
import math


In [2]:
rootdir = f'{os.getcwd()}/data/'
case = 'chb01'

file_path = f'{rootdir}{case}/{case}-summary.txt'


In [3]:
def summary(patient):
    rootdir = f'{os.getcwd()}/data/'
    case = patient
    file_path = f'{rootdir}{case}/{case}-summary.txt'
    file_metadata = []
    with open(file_path) as f:
        content_str = f.read()
        regex = re.compile(r'^\Z|\*+') # match empty string or literal asterisks
        filtered = [x for x in content_str.split('\n') if not regex.search(x)]
        regex = re.compile('Channel \d+') # match channel numbers
        channels = [x.split(':')[-1].strip() for x in filtered if regex.search(x)]
        regex = re.compile('Data Sampling Rate:')
        fs = int([x.split(':')[-1].strip() for x in filtered if regex.search(x)][0].split(' ')[0])
        regex = re.compile('^(?!Channel|Data).') # match file names
        file_metas = [x for x in filtered if regex.findall(x)]
        file_meta = {}
        for x in file_metas:
            k, v = x.partition(':')[::2]

            if k == 'Seizure Start Time':
                file_meta['Seizure Start Time'] = v
            if k == 'Seizure End Time':
                file_meta['Seizure End Time'] = v
                tup_meta = {'File Name': file_meta['File Name'], 
                                'File Start Time': file_meta['File Start Time'], 
                                'File End Time': file_meta['File End Time'],
                                'Number of Seizures in File': file_meta['Number of Seizures in File'],
                                'Seizure Start Time': file_meta['Seizure Start Time'],
                                'Seizure End Time': file_meta['Seizure End Time']
                            }
                file_metadata.append(tup_meta)

            if k == 'File Name':
                file_meta['File Name'] = v.strip()
            if k == 'File Start Time':
                file_meta['File Start Time'] = v.strip()
            if k == 'File End Time':
                file_meta['File End Time'] = v.strip()
            if k == 'Number of Seizures in File':
                if int(v) == 0:
                    if 'Seizure End Time' in file_meta:
                        del file_meta['Seizure End Time']
                    if 'Seizure Start Time' in file_meta:
                        del file_meta['Seizure Start Time']
                    file_meta['Number of Seizures in File'] = 0
                    tup_meta = {'File Name': file_meta['File Name'], 
                                'File Start Time': file_meta['File Start Time'], 
                                'File End Time': file_meta['File End Time'],
                                'Number of Seizures in File': file_meta['Number of Seizures in File']
                            }
                    file_metadata.append(tup_meta)
                if int(v) > 0:
                    file_meta['Number of Seizures in File'] = int(v.strip())

In [51]:
df = pd.DataFrame({"Seizure File": [], "Seizure Start":[], "Seizure End":[]})
temp_ss = []
temp_es = []
temp_file = []

for i in range(len(file_metadata)):
    
    if file_metadata[i]['Number of Seizures in File'] > 0:
        temp_ss.append(int(re.findall('\d+', file_metadata[i]['Seizure Start Time'])[-1]))
        temp_es.append(int(re.findall('\d+', file_metadata[i]['Seizure End Time'])[-1]))
        temp_file.append(file_metadata[i]['File Name'].replace('.edf',''))
        
        
for i in range(len(temp_ss)):
    df.loc[len(df)] = [temp_file[i], temp_ss[i], temp_es[i]]      

df['Seizure Duration'] = df.apply(lambda x: x['Seizure End'] - x['Seizure Start'], axis=1)
df

ValueError: Wrong number of items passed 3, placement implies 1

In [6]:
def feature_vector(data, times):

    sampling_freq = data.info['sfreq']
    start_stop_seconds = np.array(times)
    window_size = start_stop_seconds[1]-start_stop_seconds[0]
    start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
    eeg_channel_indices = mne.pick_types(data.info, meg=False, eeg=True)
    eeg_data, times = data[eeg_channel_indices, start_sample:stop_sample]

    coeffs = pywt.wavedec(eeg_data[0:23], "coif3", level=7)
    cA7, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
    
    band1_mean,band2_mean,band3_mean,band4_mean,band5_mean,band6_mean = [],[],[],[],[],[] # mean
    band1_std,band2_std,band3_std,band4_std,band5_std,band6_std = [],[],[],[],[],[] # standard deviation
    band1_en, band2_en, band3_en, band4_en, band5_en, band6_en = [],[],[],[],[],[] # energy
    band1_max, band2_max, band3_max, band4_max, band5_max, band6_max = [],[],[],[],[],[] # max 
    band1_min, band2_min, band3_min, band4_min, band5_min, band6_min = [],[],[],[],[],[] # min
    band1_skew, band2_skew, band3_skew, band4_skew, band5_skew, band6_skew = [],[],[],[],[],[] #skewness

    # Calculate 6 features of DWT coefficients.
    for i in range(len(cD1)):
        band1_en.append(np.sum(cD7[i, :] ** 2))
        band2_en.append(np.sum(cD6[i, :] ** 2))
        band3_en.append(np.sum(cD5[i, :] ** 2))
        band4_en.append(np.sum(cD4[i, :] ** 2))
        band5_en.append(np.sum(cD3[i, :] ** 2))
        band6_en.append(np.sum(cD2[i, :] ** 2))

        band1_max.append(np.max(cD7[i, :]))
        band2_max.append(np.max(cD6[i, :]))
        band3_max.append(np.max(cD5[i, :]))
        band4_max.append(np.max(cD4[i, :]))
        band5_max.append(np.max(cD3[i, :]))
        band6_max.append(np.max(cD2[i, :]))

        band1_min.append(np.min(cD7[i, :]))
        band2_min.append(np.min(cD6[i, :]))
        band3_min.append(np.min(cD5[i, :]))
        band4_min.append(np.min(cD4[i, :]))
        band5_min.append(np.min(cD3[i, :]))
        band6_min.append(np.min(cD2[i, :]))

        band1_mean.append(np.mean(cD7[i, :]))
        band2_mean.append(np.mean(cD6[i, :]))
        band3_mean.append(np.mean(cD5[i, :]))
        band4_mean.append(np.mean(cD4[i, :]))
        band5_mean.append(np.mean(cD3[i, :]))
        band6_mean.append(np.mean(cD2[i, :]))

        band1_std.append(np.std(cD7[i, :]))
        band2_std.append(np.std(cD6[i, :]))
        band3_std.append(np.std(cD5[i, :]))
        band4_std.append(np.std(cD4[i, :]))
        band5_std.append(np.std(cD3[i, :]))
        band6_std.append(np.std(cD2[i, :]))

        band1_skew.append(skew(cD7[i, :]))
        band2_skew.append(skew(cD6[i, :]))
        band3_skew.append(skew(cD5[i, :]))
        band4_skew.append(skew(cD4[i, :]))
        band5_skew.append(skew(cD3[i, :]))
        band6_skew.append(skew(cD2[i, :]))

    band1_en = (np.array(band1_en).reshape(1, -1))
    band2_en = (np.array(band2_en).reshape(1, -1))
    band3_en = (np.array(band3_en).reshape(1, -1))
    band4_en = (np.array(band4_en).reshape(1, -1))
    band5_en = (np.array(band5_en).reshape(1, -1))
    band6_en = (np.array(band6_en).reshape(1, -1))

    band1_max = np.array(band1_max).reshape(1, -1)
    band2_max = np.array(band2_max).reshape(1, -1)
    band3_max = np.array(band3_max).reshape(1, -1)
    band4_max = np.array(band4_max).reshape(1, -1)
    band5_max = np.array(band5_max).reshape(1, -1)
    band6_max = np.array(band6_max).reshape(1, -1)

    band1_min = np.array(band1_min).reshape(1, -1)
    band2_min = np.array(band2_min).reshape(1, -1)
    band3_min = np.array(band3_min).reshape(1, -1)
    band4_min = np.array(band4_min).reshape(1, -1)
    band5_min = np.array(band5_min).reshape(1, -1)
    band6_min = np.array(band6_min).reshape(1, -1)

    band1_mean = np.array(band1_mean).reshape(1, -1)
    band2_mean = np.array(band2_mean).reshape(1, -1)
    band3_mean = np.array(band3_mean).reshape(1, -1)
    band4_mean = np.array(band4_mean).reshape(1, -1)
    band5_mean = np.array(band5_mean).reshape(1, -1)
    band6_mean = np.array(band6_mean).reshape(1, -1)

    band1_std = np.array(band1_std).reshape(1, -1)
    band2_std = np.array(band2_std).reshape(1, -1)
    band3_std = np.array(band3_std).reshape(1, -1)
    band4_std = np.array(band4_std).reshape(1, -1)
    band5_std = np.array(band5_std).reshape(1, -1)
    band6_std = np.array(band6_std).reshape(1, -1)

    band1_skew = np.array(band1_skew).reshape(1, -1)
    band2_skew = np.array(band2_skew).reshape(1, -1)
    band3_skew = np.array(band3_skew).reshape(1, -1)
    band4_skew = np.array(band4_skew).reshape(1, -1)
    band5_skew = np.array(band5_skew).reshape(1, -1)
    band6_skew = np.array(band6_skew).reshape(1, -1)

    feature_vector = np.concatenate((band1_en, band1_max, band1_min, band1_mean, band1_std, band1_skew,
                                    band2_en, band2_max, band2_min, band2_mean, band2_std, band2_skew,
                                    band3_en, band3_max, band3_min, band3_mean, band3_std, band3_skew,
                                    band4_en, band4_max, band4_min, band4_mean, band4_std, band4_skew,
                                    band5_en, band5_max, band5_min, band5_mean, band5_std, band5_skew,
                                    band6_en, band6_max, band6_min, band6_mean, band6_std, band6_skew
                                        ), axis=0)

    # feature vector if 23 x 36 
    return feature_vector 

# Create wavelet features for 3 sec epochs of ictal-states

In [43]:
overall_features = []
seconds = 3

for i in range(df.shape[0]):
    time = [df['Seizure Start'][i], df['Seizure End'][i]]
    filename = df['Seizure File'][i]
    
    for j in range(int((math.floor(df["Seizure Duration"][i])/seconds))):
        data = mne.io.read_raw_edf(input_fname=f'{rootdir}{case}/{filename}.edf', preload=False, verbose='Error')
        times = [df['Seizure Start'][i] + j*seconds, df['Seizure Start'][i] +j*seconds + seconds]
        overall_features.append(feature_vector(data, times)) 
        

overall_features = np.reshape(overall_features, (146,23*36))
np.shape(overall_features)  

savename =  case + '_feature_ictal' + '.npy'
dir = os.path.join(rootdir+case+"/feature_ictal")
    
if not os.path.exists(dir):
    os.mkdir(dir)
            
savepath = f'{dir}/{savename}'
np.save(savepath, overall_features)  

savename =  case + '_feature_ictal' + '.npy'
dir = os.path.join(rootdir+case+"/feature_ictal")
    
if not os.path.exists(dir):
    os.mkdir(dir)
            
savepath = f'{dir}/{savename}'
np.save(savepath, overall_features)  

In [45]:
np.shape(overall_features)[0]

42

# Create wavelet features for 3 sec epochs of inter-ictal states

In [47]:
overall_features = []
seconds = 3

for i in range(df.shape[0]):
    time = [df['Seizure Start'][i], df['Seizure End'][i]]
    filename = df['Seizure File'][i]
    
    for j in range(6):
        data = mne.io.read_raw_edf(input_fname=f'{rootdir}{case}/{filename}.edf', preload=False, verbose='Error')
        times = [df['Seizure End'][i] + j*seconds, df['Seizure End'][i] +j*seconds + seconds]
        overall_features.append(feature_vector(data, times)) 
        
overall_features = np.reshape(overall_features, (np.shape(overall_features)[0],23*36))
np.shape(overall_features)  

savename = case +'_feature_interictal' + '.npy'
dir = os.path.join(rootdir+case+"/feature_interictal")
    
if not os.path.exists(dir):
    os.mkdir(dir)
            
savepath = f'{dir}/{savename}'
np.save(savepath, overall_features)    
        

/Users/martinnguyen/opt/anaconda3/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


# Generate Ictal interval of 10 sec

In [21]:
for i in range(df.shape[0]):
    marker_start = df['Seizure Start'][i]
    marker_end = df['Seizure End'][i]
    filename = df['Seizure File'][i]
    
    data = mne.io \
            .read_raw_edf(input_fname=f'{rootdir}{case}/{filename}.edf', preload=False, verbose='Error') \
            .crop(tmin=marker_start+20-2, tmax=marker_start+20+2) \
            .get_data(picks='all', units='uV', return_times=False)
    savename = filename.split('.')[0] + '_ictal' + '.npy'
    dir = os.path.join(rootdir+case+"/ictal")
    
    if not os.path.exists(dir):
        os.mkdir(dir)
            
    savepath = f'{dir}/{savename}'
    np.save(savepath, data)

In [42]:
np.shape(np.load(f'{rootdir}{case}/ictal/chb01_04_ictal.npy'))

(23, 6913)

### Generate pre-ictal interval of 10 sec

In [22]:
for i in range(df.shape[0]):
    marker_start = df['Seizure Start'][i]
    filename = df['Seizure File'][i]
    seconds = 2
    
    data = mne.io \
            .read_raw_edf(input_fname=f'{rootdir}{case}/{filename}.edf', preload=False, verbose='Error') \
            .crop(tmin=marker_start-6*seconds, tmax=marker_start-6*seconds+2) \
            .get_data(picks='all', units='uV', return_times=False)
            
    savename = filename.split('.')[0] + '_preictal10s' + '.npy'
    dir = os.path.join(rootdir+case+"/preictal_10s")
    
    if not os.path.exists(dir):
        os.mkdir(dir)
            
    savepath = f'{dir}/{savename}'
    np.save(savepath, data)

In [37]:
os.chdir(rootdir+case+"/feature_ictal")


In [38]:
x = np.load("chb01_feature_ictal.npy")
np.shape(x)

(146, 36, 23)

In [17]:
np.shape(np.load(f'{rootdir}{case}/preictal_10s/chb01_04_preictal10s.npy'))

(23, 513)